# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# basic libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2025-03-23 20:33:52.514702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file_path = "tic-tac-toe.csv"
data = pd.read_csv(file_path)
df = data.copy()


In [3]:
# convert types
target = df["class"].astype(int)
new = df.drop("class", axis=1)
# Map categorical values to numbers
mapping = {'x': 1, 'o': -1, 'b': 0}
xob = new.applymap(lambda mark: mapping[mark]) 
df = pd.concat([xob, target], axis=1)

In [4]:
y = df["class"] #separate output
X = df.drop('class', axis=1)

In [5]:
#normalise the input data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [6]:
# split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42 )

In [7]:
# create a sequential model
model = Sequential()

In [8]:
# adding several layers
model.add(Dense(9, activation='relu', input_shape = (9,)))  # Input layer
model.add(Dense(16, activation='relu'))   # Hidden layer
model.add(Dense(32, activation='relu'))   # Hidden layer
model.add(Dense(2, activation='softmax'))  # Output layer for classification

In [9]:
# compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train)

21/21 [==============================] - 1s 1ms/step - loss: 0.6683 - accuracy: 0.5687


In [11]:
# evaluating the neural network model with test data
val_loss, val_acc = model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 1ms/step - loss: 0.6239 - accuracy: 0.6840


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [12]:
model.save('tic_tac_toe')
new_model = tf.keras.models.load_model('tic_tac_toe')


INFO:tensorflow:Assets written to: tic_tac_toe/assets


INFO:tensorflow:Assets written to: tic_tac_toe/assets


In [13]:
predictions = new_model.predict([X_test])

9/9 [==============================] - 0s 1ms/step


In [14]:
n = 0
predictions_list = []
while n < len(predictions):
    predictions_list.append(np.argmax(predictions[n]))
    n += 1

predictions_series = pd.Series(predictions_list)

In [15]:
eval_df = pd.DataFrame({"actual":y_test,"pred":predictions_series})
eval_df['match'] = eval_df['actual'].fillna(-1) == eval_df['pred']
eval_df.match.value_counts()

match
False    429
True      69
Name: count, dtype: int64

Predictions are very bad.

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

### Try different amounts of layers

In [16]:
# more layers
model2 = Sequential()
# Add layers one by one
model2.add(Dense(9, activation='relu', input_shape = (9,)))  # Input layer
model2.add(Dense(16, activation='relu'))   # Hidden layer
model2.add(Dense(32, activation='relu'))   # Hidden layer
model2.add(Dense(32, activation='relu'))   # Hidden layer
model2.add(Dense(32, activation='relu'))   # Hidden layer
model2.add(Dense(32, activation='relu'))   # Hidden layer
model2.add(Dense(32, activation='relu'))   # Hidden layer
model2.add(Dense(2, activation='softmax'))  # Output layer for classification
# compiling the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train)
# evaluating the neural network model with test data
val_loss2, val_acc2 = model2.evaluate(X_test, y_test)

9/9 [==============================] - 0s 1ms/step - loss: 0.6432 - accuracy: 0.6701


### Custom Adam with updated learning rate

In [17]:
# adjust learning rate
model4 = Sequential()
# Add layers one by one
model4.add(Dense(9, activation='relu', input_shape = (9,)))  # Input layer
model4.add(Dense(16, activation='relu'))   # Hidden layer
model4.add(Dense(32, activation='relu'))   # Hidden layer
model4.add(Dense(32, activation='relu'))   # Hidden layer
model4.add(Dense(32, activation='relu'))   # Hidden layer
model4.add(Dense(32, activation='relu'))   # Hidden layer
model4.add(Dense(32, activation='relu'))   # Hidden layer
model4.add(Dense(2, activation='softmax'))  # Output layer for classification
# custom adam
adam4 = tf.keras.optimizers.Adam(
    learning_rate=0.009,
)
# compiling the model
model4.compile(optimizer=adam4, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model4.fit(X_train, y_train)
# evaluating the neural network model with test data
val_loss4, val_acc4 = model4.evaluate(X_test, y_test)

9/9 [==============================] - 0s 1ms/step - loss: 0.5684 - accuracy: 0.6736


### Adjust Epoch

In [18]:
# adust the number of epoch when you fit the training data to the model
model5 = Sequential()
# Add layers one by one
model5.add(Dense(9, activation='relu', input_shape = (9,)))  # Input layer
model5.add(Dense(16, activation='relu'))   # Hidden layer
model5.add(Dense(32, activation='relu'))   # Hidden layer
model5.add(Dense(32, activation='relu'))   # Hidden layer
model5.add(Dense(32, activation='relu'))   # Hidden layer
model5.add(Dense(32, activation='relu'))   # Hidden layer
model5.add(Dense(32, activation='relu'))   # Hidden layer
model5.add(Dense(2, activation='softmax'))  # Output layer for classification
# custom adam
adam4 = tf.keras.optimizers.Adam(
    learning_rate=0.009,
)
# compiling the model
model5.compile(optimizer=adam4, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model5.fit(X_train, y_train, epochs=10)
# evaluating the neural network model with test data
val_loss5, val_acc5 = model5.evaluate(X_test, y_test)

Epoch 1/10
21/21 [==============================] - 1s 2ms/step - loss: 0.6288 - accuracy: 0.6358
Epoch 2/10
21/21 [==============================] - 0s 1ms/step - loss: 0.5618 - accuracy: 0.6910
Epoch 3/10
21/21 [==============================] - 0s 1ms/step - loss: 0.5006 - accuracy: 0.7343
Epoch 4/10
21/21 [==============================] - 0s 1ms/step - loss: 0.4569 - accuracy: 0.7955
Epoch 5/10
21/21 [==============================] - 0s 1ms/step - loss: 0.3628 - accuracy: 0.8507
Epoch 6/10
21/21 [==============================] - 0s 1ms/step - loss: 0.2929 - accuracy: 0.8821
Epoch 7/10
21/21 [==============================] - 0s 1ms/step - loss: 0.2232 - accuracy: 0.9015
Epoch 8/10
21/21 [==============================] - 0s 1ms/step - loss: 0.1802 - accuracy: 0.9328
Epoch 9/10
21/21 [==============================] - 0s 1ms/step - loss: 0.1448 - accuracy: 0.9537
Epoch 10/10
9/9 [==============================] - 0s 1ms/step - loss: 0.1077 - accuracy: 0.9618


**Which approach(es) did you find helpful to improve your model performance?**

I found adjusting epochs made the biggest jump, but adjusting learning rate was also very helpful. The lowest increase I got by adjust the layers.